In [1]:
library(yaml)
library(COMBINEHarmonizer)
library(readxl)
library(dplyr)
library(data.table)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





Attaching package: ‘data.table’




The following objects are masked from ‘package:dplyr’:

    between, first, last




## 00. Read Config

In [2]:
root_dir <- "../"

cfg_filename <- file.path(root_dir, "config.yaml")
cfg <- yaml::read_yaml(cfg_filename)

In [3]:
input_dir <- file.path(cfg$out_dir, "stats")
out_dir <- file.path(cfg$out_dir, "stats")

# if (!file.exists(out_dir)) {
#     dir.create(out_dir, recursive = TRUE)
# }

data_dict_filename <- file.path(root_dir, COMBINEHarmonizer::DATA_DICTIONARY_EXCEL)

In [4]:
df_data_dict_main <- COMBINEHarmonizer::load_data_dict(
    data_dict_filename,
    COMBINEHarmonizer::SHEET_MAIN
)
df_data_dict_followup <- COMBINEHarmonizer::load_data_dict(
    data_dict_filename,
    COMBINEHarmonizer::SHEET_FOLLOW_UP
)
df_data_dict_analysis <- COMBINEHarmonizer::load_data_dict(
    data_dict_filename,
    COMBINEHarmonizer::SHEET_DERIVED_DATA
)

In [5]:
data_dict_main_map <- df_data_dict_main[[COMBINEHarmonizer::VAR_TYPE]]
names(data_dict_main_map) <- df_data_dict_main[[COMBINEHarmonizer::VAR_NAME]]

data_dict_followup_map <- df_data_dict_followup[[COMBINEHarmonizer::VAR_TYPE]]
names(data_dict_followup_map) <- df_data_dict_followup[[COMBINEHarmonizer::VAR_NAME]]

data_dict_analysis_map <- df_data_dict_analysis[[COMBINEHarmonizer::VAR_TYPE]]
names(data_dict_analysis_map) <- df_data_dict_analysis[[COMBINEHarmonizer::VAR_NAME]]

DATA_DICT_MAP <- list()
DATA_DICT_MAP[[COMBINEHarmonizer::SHEET_MAIN]] <- data_dict_main_map
DATA_DICT_MAP[[COMBINEHarmonizer::SHEET_FOLLOW_UP]] <- data_dict_followup_map
DATA_DICT_MAP[[COMBINEHarmonizer::SHEET_DERIVED_DATA]] <- data_dict_analysis_map

In [6]:
FILENAME_INFO_DATA_DICT_MAP <- list()
for (filename_info in COMBINEHarmonizer::FILENAME_INFOS) {
    each_name <- filename_info$name
    each_data_dict <- filename_info$data_dict
    FILENAME_INFO_DATA_DICT_MAP[[each_name]] <- DATA_DICT_MAP[[each_data_dict]]
}

PREFIX_DATA_DICT_MAP <- list()
for (each_filename in names(FILENAME_INFO_DATA_DICT_MAP)) {
    each_val <- FILENAME_INFO_DATA_DICT_MAP[[each_filename]]
    each_flatten_prefix <- COMBINEHarmonizer::flatten_filename_prefix(each_filename)
    PREFIX_DATA_DICT_MAP[[each_flatten_prefix]] <- each_val
}

## Load flatten

In [7]:
filename <- file.path(input_dir, "zz-merged-flatten-inv.csv")


In [8]:
df_inv <- read.csv(filename)

### 03. Construct df_inv

In [9]:
columns <- names(df_inv)
valid_columns <- c()
for (column in columns) {
    column_tuple <- COMBINEHarmonizer::flatten_column_tuple(column)
    prefix <- column_tuple[[1]]
    var_name <- column_tuple[[2]]
    if (!prefix %in% names(PREFIX_DATA_DICT_MAP)) {
        message(c("[WARN] not in PREFIX_DATA_DICT_MAP: column: ", column, " prefix: ", prefix))
        next
    }

    data_dict_map <- PREFIX_DATA_DICT_MAP[[prefix]]
    if (!var_name %in% names(data_dict_map)) {
        message(c("[WARN] not in DATA_DICT_MAP: var_name: ", var_name, " prefix: ", prefix))
        next
    }

    valid_columns <- c(valid_columns, column)
}

df_inv_valid <- df_inv[, valid_columns, drop = FALSE]

[WARN] not in PREFIX_DATA_DICT_MAP: column: X_study prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: center prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: subjectID prefix: 



[WARN] not in PREFIX_DATA_DICT_MAP: column: uniqueID prefix: 



[WARN] not in DATA_DICT_MAP: var_name: MRI_ID prefix: 03-05



...

[WARN] not in DATA_DICT_MAP: var_name: maternalHemorrhage prefix: 31-04



[WARN] not in DATA_DICT_MAP: var_name: maternalTrauma prefix: 31-04



[WARN] not in DATA_DICT_MAP: var_name: maternalCardioRespiratoryArrest prefix: 31-04



[WARN] not in DATA_DICT_MAP: var_name: maternalSeizure prefix: 31-04



[WARN] not in DATA_DICT_MAP: var_name: deliveryMode prefix: 31-06



In [10]:
valid_columns

[1] "X01.02.siteID"                                                      
   [2] "X01.02.birthNumber"                                                 
   [3] "X01.02.screenComment"                                               
   [4] "X01.02.coreTempLess32p5COverEq2Hr_e"                                
   [5] "X01.02.coreTempLess33p5COver1Hr_e"                                  
   [6] "X01.02.coreTempLess34COver1Hr_e"                                    
   [7] "X01.02.first6HrCoolByClinicalProtocol_e"                            
   [8] "X01.02.chromosomalAbnormality_e"                                    
   [9] "X01.02.majorCongenitalAnomaly_e"                                    
  [10] "X01.02.birthWeightLessEq1800g_e"                                    
  [11] "X01.02.infantUnlikelySurvive_e"                                     
  [12] "X01.02.first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e"
  [13] "X01.02.postnatalAgeLess6HrOrGreater24Hr_e"                          
  [14] "X01.02.enrolledConflictingTrial_e"                                  
  [15] "X01.02.first60MinAnyBloodGasPHLessEq7_i"                            
  [16] "X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i"        
  [17] "X01.02.historyPerinatalEvent_i"                                     
  [18] "X01.02.at10MinApgarLessEq5OrVent_i"                                 
  [19] "X01.02.randomEligible"                                              
  [20] "X01.02.consentStatus"                                               
  [21] "X01.02.noConsentReason"                                             
  [22] "X01.02.noInStudyReason"                                             
  [23] "X01.02.random"                                                      
  [24] "X01.02.noRandomReason"                                              
  [25] "X01.02.noRandomReasonText"                                          
  [26] "X01.02.randomDate"                                                  
  [27] "X01.02.randomTime"                                                  
  [28] "X01.02.randomNumber"                                                
  [29] "X01.02.randomTreatmentAssign"                                       
  [30] "X01.02.randomTreatmentReceive"                                      
  [31] "X01.02.treatmentBlanketType"                                        
  [32] "X01.02.inOtherTrial"                                                
  [33] "X01.02.inOtherTrialText"                                            
  [34] "X01.02.birthDate"                                                   
  [35] "X01.03.motherAge_year"                                              
  [36] "X01.03.motherRace"                                                  
  [37] "X01.03.motherRaceOther1"                                            
  [38] "X01.03.motherRaceOther2"                                            
  [39] "X01.03.motherRaceOther3"                                            
  [40] "X01.03.motherRaceOther4"                                            
  [41] "X01.03.motherRaceOther5"                                            
  [42] "X01.03.motherRaceOther6"                                            
  [43] "X01.03.motherEthnicity"                                             
  [44] "X01.03.motherMaritalStatus"                                         
  [45] "X01.03.motherEducation"                                             
  [46] "X01.03.motherInsurance"                                             
  [47] "X01.04.gravida"                                                     
  [48] "X01.04.parity"                                                      
  [49] "X01.04.multipleBirth"                                               
  [50] "X01.04.numFetus"                                                    
  [51] "X01.04.prenatalCare"                                                
  [52] "X01.04.hypertensionEclampsia"                                       

In [11]:
length(names(df_inv_valid))

[1] 6106

## 03. all

In [12]:
y_columns <- c(
    "X30.02.moderateSevereDisabilityOrDeath",
    "X30.02.disabilityLevel",
    "X31.05.disabilityLevelDeath",

    "X20.04.BayleyIIICognitiveComposite",
    "X20.04.BayleyIIILanguageComposite",
    "X20.04.BayleyIIIMotorComposite"
)

value_columns <- c(
    "corr",
    "r2",
    "pvalue",
    "valid",
    "valid_x",
    "valid_y",
    "valid_percent"
)

to_csv_column <- function(y_column, value_column) {
    paste(c(y_column, value_column), collapse = " ")
}

reorder_columns <- function(y_columns, value_columns) {
    ret <- c()
    for (y_column in y_columns) {
        for (value_column in value_columns) {
            ret <- c(ret, to_csv_column(y_column, value_column))
        }
    }

    return(ret)
}

reordered_columns <- reorder_columns(y_columns, value_columns)
reordered_columns

[1] "X30.02.moderateSevereDisabilityOrDeath corr"         
 [2] "X30.02.moderateSevereDisabilityOrDeath r2"           
 [3] "X30.02.moderateSevereDisabilityOrDeath pvalue"       
 [4] "X30.02.moderateSevereDisabilityOrDeath valid"        
 [5] "X30.02.moderateSevereDisabilityOrDeath valid_x"      
 [6] "X30.02.moderateSevereDisabilityOrDeath valid_y"      
 [7] "X30.02.moderateSevereDisabilityOrDeath valid_percent"
 [8] "X30.02.disabilityLevel corr"                         
 [9] "X30.02.disabilityLevel r2"                           
[10] "X30.02.disabilityLevel pvalue"                       
[11] "X30.02.disabilityLevel valid"                        
[12] "X30.02.disabilityLevel valid_x"                      
[13] "X30.02.disabilityLevel valid_y"                      
[14] "X30.02.disabilityLevel valid_percent"                
[15] "X31.05.disabilityLevelDeath corr"                    
[16] "X31.05.disabilityLevelDeath r2"                      
[17] "X31.05.disabilityLevelDeath pvalue"                  
[18] "X31.05.disabilityLevelDeath valid"                   
[19] "X31.05.disabilityLevelDeath valid_x"                 
[20] "X31.05.disabilityLevelDeath valid_y"                 
[21] "X31.05.disabilityLevelDeath valid_percent"           
[22] "X20.04.BayleyIIICognitiveComposite corr"             
[23] "X20.04.BayleyIIICognitiveComposite r2"               
[24] "X20.04.BayleyIIICognitiveComposite pvalue"           
[25] "X20.04.BayleyIIICognitiveComposite valid"            
[26] "X20.04.BayleyIIICognitiveComposite valid_x"          
[27] "X20.04.BayleyIIICognitiveComposite valid_y"          
[28] "X20.04.BayleyIIICognitiveComposite valid_percent"    
[29] "X20.04.BayleyIIILanguageComposite corr"              
[30] "X20.04.BayleyIIILanguageComposite r2"                
[31] "X20.04.BayleyIIILanguageComposite pvalue"            
[32] "X20.04.BayleyIIILanguageComposite valid"             
[33] "X20.04.BayleyIIILanguageComposite valid_x"           
[34] "X20.04.BayleyIIILanguageComposite valid_y"           
[35] "X20.04.BayleyIIILanguageComposite valid_percent"     
[36] "X20.04.BayleyIIIMotorComposite corr"                 
[37] "X20.04.BayleyIIIMotorComposite r2"                   
[38] "X20.04.BayleyIIIMotorComposite pvalue"               
[39] "X20.04.BayleyIIIMotorComposite valid"                
[40] "X20.04.BayleyIIIMotorComposite valid_x"              
[41] "X20.04.BayleyIIIMotorComposite valid_y"              
[42] "X20.04.BayleyIIIMotorComposite valid_percent"

In [13]:
.GlobalEnv$EXCLUDE_COLUMNS <- c(y_columns,
    "X01.01.birthNumber", # 1: 525 2: 7
    "X01.01.coreTempBelow32p5COver2H", # 0: 532 (fit criteria),
    "X01.07.pre_CoolbyIceGelPack", # 0: 250, 1: 2

    "X30.01.blindness", # part of the outcome definition
    "X30.01.multipleImpairment", # part of the outcome defition
    "X30.01.moderateSevereCerebralPalsy", # part of the outcome defition
    "X30.01.hearingImpairedWithAid", # part of the outcome defition
    "X30.01.cerebralPalsy", # part of the outcome defition
    "X30.01.afterDischargeSeizure", # part of the outcome defition
    "X30.01.grossMotorFunctionLevel", # part of the outcome defition

    "X30.01.ageDeath_day", # related t the outcome definition

    "X30.02.normalPrimaryOutcome", # part of the outcome defition
    "X30.02.moderateSevereDisabilitySurvivor", # part of the outcome defition
    "X30.02.disabilityLevelSurvivor", # part of the outcome defition
    "X30.02.disabilityLevel", # part of the outcome defition
    "X30.02.outcomeGroup" # part of the outcome defition
)

.GlobalEnv$INCLUDE_COLUMN_PREFIX <- c(
    "01-02",
    "01-03",
    "01-04",
    "01-05",
    "01-06",
    "01-07",
    "01-08",
    "01-09",
    "01-10",
    "01-12",
    "03-04",
    "03-05",
    "04-01",
    "04-02",
    "04-03",
    "04-04",
    "04-05",
    "04-06",
    "04-07",
    "04-08",
    "04-09",
    "04-10",
    "04-11",
    "04-12",
    "04-13",
    "04-14",
    "04-15",
    "30-01",
    "30-02",
    "30-03",
    "31-02",
    "31-03",
    "31-04",
    "31-05",
    "31-06",
    "31-07"
)

In [14]:
valid_column <- function(df, column) {
    if (column %in% .GlobalEnv$EXCLUDE_COLUMNS) {
        return(FALSE)
    }

    column_tuple <- COMBINEHarmonizer::flatten_column_tuple(column)

    column_prefix <- column_tuple[[1]]
    var_name <- column_tuple[[2]]

    if (!column_prefix %in% .GlobalEnv$INCLUDE_COLUMN_PREFIX) {
        return(FALSE)
    }

    if (length(grep(pattern = "Bayley", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "death", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "wdrawSupport", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "limitCare", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    if (length(grep(pattern = "\\.followupCenter$", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    # XXX remove Date
    if (length(grep(pattern = "Date\\d*$", x = column, perl = TRUE)) != 0) {
        return(FALSE)
    }

    # XXX remove Time, but retain spontaneousRespirationTime
    if (length(grep(pattern = "Time\\d*$", x = column, perl = TRUE) != 0) &&
            length(grep(pattern = "spontaneousRespirationTime", x = column, perl = TRUE)) == 0) {
        return(FALSE)
    }

    # with only MRINRNPatternOfInjury in MRI.
    if (column_prefix == "03-05") {
        if (var_name != "MRINRNPatternOfInjury") {
            return(FALSE)
        }
    }

    if (class(df[[column]]) != "numeric") {
        message(c("[WARN] invalid dtype: column: ", column, " type: ", typeof(df[column])))
        return(FALSE)
    }

    sum_is_na_column <- sum(!is.na(df[column]))
    if (sum_is_na_column < 20) {
        message(c("[WARN] to few valid samples: column: ", column, " count: ", sum_is_na_column))
        return(FALSE)
    }

    the_list <- df[[column]]
    length_unique <- length(unique(the_list[!is.na(the_list)]))
    if (length_unique <= 1) {
        print(c("[WARN] constant column: ", column))
        return(FALSE)
    }

    return(TRUE)
}

In [15]:
columns <- names(df_inv_valid)
x_columns <- columns[sapply(columns, \(x) valid_column(df_inv_valid, x))]
length(x_columns)

[WARN] invalid dtype: column: X01.02.siteID type: list



[WARN] invalid dtype: column: X01.02.screenComment type: list



[1] "[WARN] constant column: "            "X01.02.coreTempLess32p5COverEq2Hr_e"
[1] "[WARN] constant column: "          "X01.02.coreTempLess33p5COver1Hr_e"
[1] "[WARN] constant column: "        "X01.02.coreTempLess34COver1Hr_e"
[1] "[WARN] constant column: "               
[2] "X01.02.first6HrCoolByClinicalProtocol_e"
[1] "[WARN] constant column: "        "X01.02.chromosomalAbnormality_e"
[1] "[WARN] constant column: "        "X01.02.majorCongenitalAnomaly_e"
[1] "[WARN] constant column: "        "X01.02.birthWeightLessEq1800g_e"
[1] "[WARN] constant column: "       "X01.02.infantUnlikelySurvive_e"
[1] "[WARN] constant column: "                                           
[2] "X01.02.first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e"
[1] "[WARN] constant column: "                 
[2] "X01.02.postnatalAgeLess6HrOrGreater24Hr_e"
[1] "[WARN] constant column: "          "X01.02.enrolledConflictingTrial_e"
[1] "[WARN] constant column: "       "X01.02.historyPerinatalEvent_i"
[1] 

[WARN] invalid dtype: column: X01.02.consentStatus type: list



[WARN] invalid dtype: column: X01.02.noConsentReason type: list



...

[1] "[WARN] constant column: " "X30.03.MRINotDone"       


[WARN] to few valid samples: column: X30.03.MRIUnread count: 19



[1] "[WARN] constant column: " "X30.03.MRIAnalysis"      


[WARN] to few valid samples: column: X30.03.cerebralAtrophyGlobalLocal count: 15



[1] 329

In [16]:
x_columns

[1] "X01.02.birthNumber"                                         
  [2] "X01.02.first60MinAnyBloodGasPHLessEq7_i"                    
  [3] "X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i"
  [4] "X01.02.treatmentBlanketType"                                
  [5] "X01.02.inOtherTrial"                                        
  [6] "X01.03.motherAge_year"                                      
  [7] "X01.03.motherEducation"                                     
  [8] "X01.03.motherInsurance"                                     
  [9] "X01.04.gravida"                                             
 [10] "X01.04.parity"                                              
 [11] "X01.04.multipleBirth"                                       
 [12] "X01.04.prenatalCare"                                        
 [13] "X01.04.hypertensionEclampsia"                               
 [14] "X01.04.antepartumHemorrhage"                                
 [15] "X01.04.thyroidMalfunction"                                  
 [16] "X01.04.diabetes"                                            
 [17] "X01.05.ruptureOver18Hr"                                     
 [18] "X01.05.ruptureBeforeDelivery"                               
 [19] "X01.05.labor"                                               
 [20] "X01.05.fetalDecelerate"                                     
 [21] "X01.05.cordMishap"                                          
 [22] "X01.05.uterineRupture"                                      
 [23] "X01.05.shoulderDystocia"                                    
 [24] "X01.05.placentalProblem"                                    
 [25] "X01.05.maternalHemorrhage"                                  
 [26] "X01.05.maternalTrauma"                                      
 [27] "X01.05.maternalCardioRespiratoryArrest"                     
 [28] "X01.05.maternalSeizure"                                     
 [29] "X01.05.pyrexiaOver37p6C"                                    
 [30] "X01.05.chorioamnionitis"                                    
 [31] "X01.05.placentalPathologyPerformed"                         
 [32] "X01.05.histologicChorioamionitis"                           
 [33] "X01.05.laborAntibiotics"                                    
 [34] "X01.06.encephalopathyLevel"                                 
 [35] "X01.06.randomInfantAge"                                     
 [36] "X01.06.birthWeight_g"                                       
 [37] "X01.06.birthLength_cm"                                      
 [38] "X01.06.birthHeadCircumference_cm"                           
 [39] "X01.06.birthGestationalAge_week"                            
 [40] "X01.06.infantSex"                                           
 [41] "X01.06.infantOutborn"                                       
 [42] "X01.06.outbornInHospital"                                   
 [43] "X01.06.outbornOutHospital"                                  
 [44] "X01.06.Apgar1min"                                           
 [45] "X01.06.Apgar5min"                                           
 [46] "X01.06.Apgar10min"                                          
 [47] "X01.06.Apgar15min"                                          
 [48] "X01.06.Apgar20min"                                          
 [49] "X01.06.deliveryResuscitation"                               
 [50] "X01.06.deliveryOxygen"                                      
 [51] "X01.06.deliveryBaggingAndMask"                              
 [52] "X01.06.deliveryChestCompression"                            
 [53] "X01.06.deliveryIntubation"                                  
 [54] "X01.06.deliveryDrug"                                        
 [55] "X01.06.at10MinContinueResuscitation"                        
 [56] "X01.06.at10MinOxygen"                                       
 [57] "X01.06.at10MinBaggingAndMask"                               
 [58] "X01.06.at10MinChestCompression"                             
 [59] "X01.06.at10MinIntubation"                          

In [17]:
y_columns

[1] "X30.02.moderateSevereDisabilityOrDeath"
[2] "X30.02.disabilityLevel"                
[3] "X31.05.disabilityLevelDeath"           
[4] "X20.04.BayleyIIICognitiveComposite"    
[5] "X20.04.BayleyIIILanguageComposite"     
[6] "X20.04.BayleyIIIMotorComposite"

In [18]:
df_corr_all <- COMBINEHarmonizer::corr(df_inv_valid, x_columns, y_columns)

df_corr_all_pivot <- data.table::dcast(
    data.table::setDT(df_corr_all),
    x ~ y,
    fun = list(mean),
    value.var = "val"
)

r_column <- "X20.04.BayleyIIICognitiveComposite pvalue"
df_corr_all_pivot[["pvalue_lt_threshold"]] <- df_corr_all_pivot[[r_column]] < COMBINEHarmonizer::P_VALUE_THRESHOLD

all_reorder_columns <- c("x", "pvalue_lt_threshold", reordered_columns)

df_corr_all_pivot_reorder <- df_corr_all_pivot[, ..all_reorder_columns] %>%
    dplyr::arrange(desc(pvalue_lt_threshold), desc(`X20.04.BayleyIIICognitiveComposite r2`))

out_filename <- file.path(out_dir, "31-02-corr-all-R.csv")
write.csv(df_corr_all_pivot_reorder, file = out_filename, row.names = FALSE)

corr: (1/329) x_column: X01.02.birthNumber



corr: (2/329) x_column: X01.02.first60MinAnyBloodGasPHLessEq7_i



corr: (3/329) x_column: X01.02.first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i



corr: (4/329) x_column: X01.02.treatmentBlanketType



corr: (5/329) x_column: X01.02.inOtherTrial



...

corr: (325/329) x_column: X31.03.MRINRNPatternOfInjuryAvg



corr: (326/329) x_column: X31.03.MRINRNPatternOfInjuryMax



corr: (327/329) x_column: X31.04.perinatalSentinelEvent



corr: (328/329) x_column: X31.06.emergencyCSection



corr: (329/329) x_column: X31.07.lengthOfStay_day

